In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import requests
import sqlite3
import json
from bs4 import BeautifulSoup
import os
import time

In [80]:
class Animal():
    def __init__(self, vars ,**kwargs):
        self.__dict__.update(vars, **kwargs)

class crawl(Animal):

    def ptt(self, url):
        header = self.__dict__
        r = requests.get(url, headers=header)

        if r.status_code == 200:
            soup = BeautifulSoup(r.text, "html.parser")
       
            web_list = []
            for i in soup.find_all("div", class_='r-ent'):
                j = i.find_all("a", href=True)
                if len(j)>0:
                    web_list.append("https://www.ptt.cc/"+j[0]["href"])

        else: return "something wrong about ptt stock %s." %url

        for j in web_list:
            r2 = requests.get(j, headers=header)
            if r2.status_code == 200:

                soup2 = BeautifulSoup(r2.text, "html.parser")
                soup2.title.string.split("-")[0]

                text = soup2.find(id="main-container").text.split("--")
                pre_text = text[0].split('\n')

                push_text= []
                for i in soup2.find_all("div", class_="push"):
                    push_text.append(i.text)
                
                s = soup2.title.string.split("-")[0]

                with open(os.path.join("news/ptt/", repr(s).replace("\\","").replace("/","").replace(" ","")+".txt"), "w") as f:
                    f.write("\n".join(pre_text[1:]))
                    f.write("\n".join(push_text))

            else: return "th post %s has something wrong" %j
    
    def udn(self, url):
        header = self.__dict__
        r = requests.get(url, headers=header)

        if r.status_code == 200:
            soup = BeautifulSoup(r.text, "html.parser")
            
            web_list = []
            for i in soup.find_all("div", class_="story__content"):
                j = i.find_all("a", href=True)
                web_list.append(j[0]["href"])

        else: return "something wrong about udn %s." %url

        for j in web_list:
            r2 = requests.get(j, headers=header)
            if r2.status_code == 200:

                soup = BeautifulSoup(r2.text, "html.parser")

                timing = soup.find("time",class_="article-body__time")
                if timing is not None:
                    
                    timing = timing.text
                    title = soup.find(id="story_art_title").text
                    body = soup.find("section",class_="article-body__editor").text

                    name = repr(title).replace("\\","").replace("/","").replace(" ","")

                    with open(os.path.join("news/udn/", name+".txt"), "w") as f:
                        f.write(title)
                        f.write('\n')
                        f.write(timing)
                        f.write(body)

            else: return "th post %s has something wrong" %j

    def digitimes(self, url, page=5, **kwargs):
        header = self.__dict__

        web_list = []
        for i in range(page+1):
            if "auth" in kwargs.keys(): 
                auth = kwargs["auth"]
            else: auth = None
            r = requests.post(url, 
                            headers=header,
                            params= {"page":[i]})
            if r.status_code == 200:
                soup = BeautifulSoup(r.text, "html.parser")
                text = soup.find_all("div",class_="subject_area")
                if text is not None:
                    for j in text:
                        web_list.append("https://www.digitimes.com.tw/"+(j.find("a").attrs["href"]))
                else: return "something wrong about digitimes %s." %url
        
        for k in web_list:
            r2 = requests.get(k, headers=header, auth=auth)
            if r2.status_code == 200:
                soup = BeautifulSoup(r2.text, "html.parser")

                title = soup.find("h1").text
                title = repr(title).replace("\\","").replace("/","").replace(" ","")

                with open(os.path.join("news/digitimes/", title+".txt"), "w") as f:
                    f.write(title.replace("'",""))
                    f.write("\n")
                    f.write(soup.find("time").text)
                    f.write("\n")
                    for i in soup.find_all("p", class_="main_p"):
                        k = repr(i.text).replace(r"\n","").replace(r"\r","").replace(r"\t","")
                        f.write(k.replace("'",""))
                        f.write("\n")
            else: return "th post %s has something wrong" %k
    
    def finace(self, url, co_id, year, dtype="AI1"):
        param = {"id":"", "key":"", "step":1,"co_id":co_id, "year":year, "seamon":"", "mtype":"A", "dtype":dtype}

        r = requests.post(url, params=param)
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, "html.parser")
            name = soup.find_all("a", href=True)

        else: return print("something wrong about this web %s." %co_id)

        if name[0].text != "":
            for j in name:
                param2 = {"step":9, "kind":"A", "co_id":co_id, "filename":j.text}
                r2 = requests.post(url, params=param2)

                if r2.status_code == 200:
                    soup2 = BeautifulSoup(r2.text, "html.parser")
                    link = soup2.find("a", href=True)

                    response = requests.get("https://doc.twse.com.tw/"+link["href"])
                    if response.status_code == 200:
                        with open("news/finance/"+j.text, "wb") as f:
                            f.write(response.content)
                    else: return print("something wrong about this finance report %s." %"https://doc.twse.com.tw/"+link["href"])
                else: return print("something wrong about this download page %s." %j.text)
        
        else:
            param2 = {"step":1, "dtype":dtype, "mtype":"A", "co_id":co_id, "check2858":"Y", "year":year}
            r2 = requests.post("https://doc.twse.com.tw/server-java/t57sb01", params=param2)
            if r2.status_code == 200:
                soup2 = BeautifulSoup(r2.text, "html.parser")
                name = soup2.find_all("a", href=True)

            for j in name:
                param3 = {"step":9, "kind":"A", "co_id":co_id, "filename":j.text}
                r3 = requests.post(url, params=param3)

                if r3.status_code == 200:
                    soup3 = BeautifulSoup(r3.text, "html.parser")
                    link = soup3.find("a", href=True)

                    response = requests.get("https://doc.twse.com.tw/"+link["href"])
                    if response.status_code == 200:
                        with open("news/finance/"+j.text, "wb") as f:
                            f.write(response.content)
                    else: return print("something wrong about this finance report %s." %"https://doc.twse.com.tw/"+link["href"])
                else: return print("something wrong about this download page %s." %j.text)


Set up the header for crawling

In [3]:
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
cookie = "uuid_tt_dd=10_20564905260-1703225841522-750867; c_first_ref=www.google.com; c_segment=3; c_adb=1; dc_sid=df9d26e5f60c9796335d2b2bdc46e0aa; hide_login=1; SidecHatdocDescBoxNum=true; creative_btn_mp=3; dc_session_id=10_1704181968889.661952; SESSION=ee7d1139-efc1-4491-8412-ffed93215f66; c_first_page=https%3A//blog.csdn.net/yinjun3215/article/details/108477055; c_dsid=11_1704184443684.220868; c_pref=https%3A//www.google.com/; c_ref=https%3A//blog.csdn.net/yinjun3215/article/details/108477055; c_utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7ECTRLIST%7ERate-1-102261567-blog-108477055.235%5Ev40%5Epc_relevant_anti_t3; c_utm_relevant_index=2; c_page_id=default; log_Id_pv=19; loginbox_strategy=%7B%22taskId%22%3A308%2C%22abCheckTime%22%3A1704163911128%2C%22version%22%3A%22exp1%22%2C%22blog-threeH-dialogtipShowTimes%22%3A8%2C%22blog-threeH-dialog%22%3A1704182063207%7D; log_Id_view=477; log_Id_click=7; dc_tos=s6mm2j"
x = dict({"User-Agent":user_agent, "Cookie":cookie})

In [81]:
test = crawl(x)

In [95]:
for k in range(6822,6823+1):
    ptt_path = "https://www.ptt.cc/bbs/Stock/index"+str(k)+".html"
    test.ptt(url=ptt_path)
    if (k%3 == 0):
        print(k)
    time.sleep(5)

6822


In [6]:
for k in range(10):
    ptt_path = "https://money.udn.com/rank/ajax_newest/1001/0/"+str(k)
    test.udn(url=ptt_path)
    if (k%2 == 0):
        print(k)
    time.sleep(5)

0
2
4
6
8


In [56]:
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
cookie = "vid=3421667e87765c5f; promotype=+; MyPwd=Br3%60ZV%7B4GJ0L48X27ZQHP0E5KN6B3UZ1HF3D1NL0PQIYV3CF3YOWL3L; RememberMyPwd=Y; MyName%5Fcn=; ASPSESSIONIDQURSQTSD=JIPDJCCCDDINKEBEFJLINEKJ; ASPSESSIONIDSUSCACAR=MAAPBGCCAGJCACCDLPADJJAK; ASPSESSIONIDAWQTBADA=EEDKNMBAAJGCCGFJFOPLKAJG; autorefresh=Y; ASPSESSIONIDCURQDBDA=CNDEFOGBJKKDIHJBHAFFMOPO; ASPSESSIONIDSERTTRRQ=DLMONBHBPNGNLNJCNMAHJOKF; ASPSESSIONIDAWSSABDA=OJDILHNBPHBHLECCOGGJIFPL; ASPSESSIONIDSEQRSTRQ=BBMMHJNBGPGFMKNELMDOKKBF; ASPSESSIONIDAWTSDCBB=PLLDHAICPDMBDKLMPHPOBCJH; ASPSESSIONIDSEQRRSQR=LPJCECICMPIHNHBOINLFOAAL; weblogIP=122A147A142A30; ASPSESSIONIDCWRQDBCB=HLGLMCGBINDIOLEFLLHLOCGB; ASPSESSIONIDQGSRTTRR=HCHOBPACIIOKDBMPNMKCHGLP; mem%5Fpro=N; justLogout=Y; ASPSESSIONIDSGTSQSRR=BLAGLGHCMGDFOOMNOFNDEJOE; MyName=%E9%BB%83%E9%9D%96%E5%87%B1; LEADtservice=%2CLEA%2CASC%2CCVC; LEAright=Y; CVC%5FEmail=james%2Ejk%2Ehuang%40foxconn%2Ecom; CVC%5FUserid=XAA45006; MemSID=661347451; ASC%5FUserid=XAA45006; yUID=james%2Ejk%2Ehuang%40foxconn%2Ecom; CVC%5FMemberid=XAB25034; MemberIDPDF=XAB25034; SSLUID=XAA45006; occupation=1; ASC%5FEmail=james%2Ejk%2Ehuang%40foxconn%2Ecom; LEAMyName=%E9%BB%83%E9%9D%96%E5%87%B1; LEAyUID=james%2Ejk%2Ehuang%40foxconn%2Ecom; LEAMemberID=XAB25034; CVC%5FMyName=%E9%BB%83%E9%9D%96%E5%87%B1; DownMYID=james%2Ejk%2Ehuang%40foxconn%2Ecom; MyID=BkN%6029nZXVd4UB3tQH3SP%2DP89VKTk7235X77jOYEYLXNZ%2FBKJ6G46OYgUK1GV3PXCAvPMLSK4XOOT7%60ZHH6FE7MFQKZoEQV9AAYZ8D3LPf9X1OG2L9DPVAXCA9HI5DP772E779EKeBJY4DIYP6QIK8TN5pMOFY072U9YJEWEDO2w4L4QG0TWARYE53GTK9dYFKY2TLPRCIN26ZM4A1nD8ZX3HH23AQK1FPOYANMoX6UQR6DR41CFMCA9G1QO8m15BRA3NKWKP8KN99FG53S5%2FS8KIZYIF5Z4UIGFWNBABHNUb28O66SJA08R9HRLTVOXXLR7QpEURWPN9XDTIVECKE7YQWZ8EVClDKV6QZ5UDE4IT0OT6CXTF43Z4O; DownUID=XAA45006; ASC%5FMyName=%E9%BB%83%E9%9D%96%E5%87%B1; ASC%5FMemberid=XAB25034; UserIDPDF=XAA45006; DownMemID=XAB25034; CVCsid=661347451; sDownImage=CAR%2DY%2CSTN%2DY%2CICM%2DY%2CICD%2DY%2CEVF%2DY%2CSVR%2DY%2CCMP%2DY%2CCSE%2DY%2CIOT%2DY%2CSMG%2DY%2CBWC%2DY%2CSML%2DY%2CSMF%2DY%2CATC%2DY%2CAAI%2DY%2CDHM%2DY%2CMCN%2DY%2CGNT%2DY%2CAIP%2DY%2CAIR%2DY%2C5GF%2DY%2CPCE%2DY%2C; sDataView=CAR%2DY%2CSTN%2DN%2CICM%2DY%2CICD%2DY%2CEVF%2DN%2CSVR%2DY%2CCMP%2DY%2CCSE%2DN%2CIOT%2DN%2CSMG%2DN%2CBWC%2DN%2CSML%2DN%2CSMF%2DY%2CATC%2DN%2CAAI%2DN%2CDHM%2DN%2CMCN%2DY%2CGNT%2DN%2CAIP%2DN%2CAIR%2DN%2C5GF%2DN%2CPCE%2DN%2C; sDownRight=CAR%2DN%2CSTN%2DN%2CICM%2DN%2CICD%2DN%2CEVF%2DN%2CSVR%2DN%2CCMP%2DN%2CCSE%2DN%2CIOT%2DN%2CSMG%2DN%2CBWC%2DN%2CSML%2DN%2CSMF%2DN%2CATC%2DN%2CAAI%2DN%2CDHM%2DN%2CMCN%2DN%2CGNT%2DN%2CAIP%2DN%2CAIR%2DN%2C5GF%2DN%2CPCE%2DN%2C; sDataFile=CAR%2DY%2CSTN%2DN%2CICM%2DY%2CICD%2DY%2CEVF%2DN%2CSVR%2DY%2CCMP%2DY%2CCSE%2DN%2CIOT%2DN%2CSMG%2DN%2CBWC%2DN%2CSML%2DN%2CSMF%2DY%2CATC%2DN%2CAAI%2DN%2CDHM%2DN%2CMCN%2DY%2CGNT%2DN%2CAIP%2DN%2CAIR%2DN%2C5GF%2DN%2CPCE%2DN%2C; sRptRight=%2CCAR%2CSTN%2CICM%2CICD%2CEVF%2CSVR%2CCMP%2CCSE%2CIOT%2CSMG%2CBWC%2CSML%2CSMF%2CATC%2CAAI%2CDHM%2CMCN%2CGNT%2CAIP%2CAIR%2C5GF%2CPCE; sShow=Y; ASPSESSIONIDCWTQCDAA=AEJOPEHCPGMCHDGOENIELCIG; AWSALBTG=/uORw842ciPxp12V2yJY6waw3E16a9wX/UcvlC7YbKEe7AHJ0w/U/b6/UTaa6rs5pSzNF51c0ocf2oHoIOB+8HsUoTSU1Ew6APaTAtj2MEplffb2OOIH2qrqW8r2iDLzo9WAvgFLmJ/JITJRj2exDGeuoWh5R000r3JfOJZt+onK55cLmTA=; AWSALBTGCORS=/uORw842ciPxp12V2yJY6waw3E16a9wX/UcvlC7YbKEe7AHJ0w/U/b6/UTaa6rs5pSzNF51c0ocf2oHoIOB+8HsUoTSU1Ew6APaTAtj2MEplffb2OOIH2qrqW8r2iDLzo9WAvgFLmJ/JITJRj2exDGeuoWh5R000r3JfOJZt+onK55cLmTA=; AWSALB=KGNQWtiqS1l0h1g/+OMPtacS7IEGs2GGILLW7NcOfuubgyLYVG0nlZc18fDniScgJpKqNwk/wtSrwsSx6E0631LCflaSIpI3p8bij12XGPGI+JuB/9GL6Ywy3Pj6; AWSALBCORS=KGNQWtiqS1l0h1g/+OMPtacS7IEGs2GGILLW7NcOfuubgyLYVG0nlZc18fDniScgJpKqNwk/wtSrwsSx6E0631LCflaSIpI3p8bij12XGPGI+JuB/9GL6Ywy3Pj6"
origin = "https://www.digitimes.com.tw"
referer = "https://www.digitimes.com.tw/tech/dt/allnewslist.asp?CnlID=99"
content_type = "application/x-www-form-urlencoded; charset=UTF-8"
x = dict({"User-Agent":user_agent, "Cookie":cookie,"Orgin":origin ,"Referer":referer, "Content-Type":content_type})

In [94]:
test.digitimes(url="https://www.digitimes.com.tw/tech/ajax/newslist_ajax.asp", 
               auth=("james.jk.huang@foxconn.com", "Br3`ZV{4GJ0L48X27ZQHP0E5KN6B3UZ1HF3D1NL0PQIYV3CF3YOWL3L"))

In [97]:
company_code = [2330, 2454, 2317, 2412, 2382, 2881, 2308, 6505, 2882, 2303, 3711, 2886, 2891, 1303, 1301, 2002, 1216, 2884, 2207, 5880, 3008, 2892, 3045, 1326, 2357, 2885, 6669, 
                2395, 3034, 5871, 2880, 2603, 2345, 3231, 4904, 2912, 2301, 3037, 1101, 2327, 2890, 2379, 2408, 3661, 3443, 4938, 5876, 2887, 2883, 1590, 2801, 6415, 2356, 2609,
                2324, 9910, 2377, 2618, 2376, 1402, 2353, 8046, 1605, 2615, 1476, 6509, 1102, 2105, 2409, 2888, 3702, 2385, 2474, 2610, 3481, 3017, 2344, 2383, 5269, 8454]

#3711

for i in [5880]:
    test.finace(url="https://doc.twse.com.tw/server-java/t57sb01",
                co_id=i, year=112)
    time.sleep(15)

ConnectionError: ('Connection aborted.', BadStatusLine('<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">\r\n'))